<a href="https://colab.research.google.com/github/syedfasihzaidi480/Fine-tune-a-Gpt-2-model-to-generate-math-riddles/blob/main/fine-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [67]:
# Full dataset creation (10 original entries + 20 additional entries)
additional_riddles = [
    {"riddle": "What number when doubled becomes three times itself minus 10?", "solution": "10"},
    {"riddle": "I am thinking of a number. If you add 15 and then divide by 4, you get 5. What is it?", "solution": "5"},
    {"riddle": "A number divided by 3 equals its double minus 15. What is the number?", "solution": "9"},
    {"riddle": "What number decreases by 8 when you subtract a quarter of it?", "solution": "32"},
    {"riddle": "If you multiply me by 5 and subtract 12, you get 48. What number am I?", "solution": "12"},
    {"riddle": "What number becomes 100 when you add 25% of itself to it?", "solution": "80"},
    {"riddle": "Twice a number plus three times itself equals 40. What's the number?", "solution": "8"},
    {"riddle": "A number divided by 5 plus 7 equals 12. What is the number?", "solution": "25"},
    {"riddle": "What number multiplied by itself gives 64 but becomes 10 when halved?", "solution": "8"},
    {"riddle": "If you take away 7 from 5 times a number, you get 38. What's the number?", "solution": "9"},
    {"riddle": "What number becomes 3 when you subtract its square root?", "solution": "9"},
    {"riddle": "A number increased by 20% becomes 60. What was the original number?", "solution": "50"},
    {"riddle": "Four times a number minus 6 equals twice the number plus 8. What is it?", "solution": "7"},
    {"riddle": "What number divided by 0.5 gives the same result as adding 12 to it?", "solution": "8"},
    {"riddle": "If you cube a number and subtract 8, you get 19. What's the number?", "solution": "3"},
    {"riddle": "A number plus its reciprocal equals 2. What is the number?", "solution": "1"},
    {"riddle": "What number becomes 1 when you calculate 3/4 of it plus 4?", "solution": "-4"},
    {"riddle": "Three consecutive even numbers sum to 30. What's the smallest one?", "solution": "8"},
    {"riddle": "A number squared minus 5 equals 20. What is the number?", "solution": "5"},
    {"riddle": "What number multiplied by 1.5 gives the same result as adding 5 to it?", "solution": "10"},
]

original_dataset = [
    {"riddle": "What number becomes zero when you subtract 15 from half of it?", "solution": "30"},
    {"riddle": "I am a number. If you double me and add 8, you get 20. What am I?", "solution": "6"},
    {"riddle": "What number increases by 5 when you divide it by 2?", "solution": "-10"},
    {"riddle": "A number when multiplied by 3 gives 21. What is half of that number?", "solution": "3.5"},
    {"riddle": "What number decreases by 12 when you subtract a third of it?", "solution": "18"},
    {"riddle": "If you add 7 to me and multiply by 2, you get 34. What number am I?", "solution": "10"},
    {"riddle": "A number's double divided by 4 equals 5. What's the number?", "solution": "10"},
    {"riddle": "What number becomes 10 when you add 3 to half of it?", "solution": "14"},
    {"riddle": "Twice a number plus 5 equals 17. What's the number?", "solution": "6"},
    {"riddle": "What number divided by 3 and then added to 7 equals 12?", "solution": "15"},
]

full_dataset = original_dataset + additional_riddles


In [68]:
# Verify total count is 30
assert len(full_dataset) == 30, f"Dataset has {len(full_dataset)} entries instead of 30"


In [69]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TrainingArguments, Trainer
from datasets import Dataset


In [70]:
# Load GPT-2 tokenizer and set padding token
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

In [71]:
# Format dataset into a string prompt for each example
formatted_data = [
    f"Riddle: {item['riddle']}\nAnswer: {item['solution']}\n\n"
    for item in full_dataset
]

In [72]:
# Create a Hugging Face dataset
dataset = Dataset.from_dict({"text": formatted_data})


In [78]:
# Tokenization function for the dataset
def tokenize_function(examples):
    # Tokenize both the riddle and the answer, then combine them
    riddle_tokens = tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128, return_tensors="pt")
    # We use the same tokenizer for the labels (targets)
    # Shift the labels by one position to the right for causal language modeling
    labels = riddle_tokens.input_ids.clone()
    # Replace padding token with -100 to ignore loss calculation on padding tokens
    labels[labels == tokenizer.pad_token_id] = -100

    riddle_tokens["labels"] = labels  # Add the labels to the tokenized output
    return riddle_tokens

tokenized_dataset = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [79]:
# Load the pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained("gpt2")


In [80]:
# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    logging_steps=100,
)

In [81]:
# Create Trainer object
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)

In [82]:

# Fine-tune the model
trainer.train()


Step,Training Loss


TrainOutput(global_step=45, training_loss=1.9594416300455728, metrics={'train_runtime': 17.1172, 'train_samples_per_second': 5.258, 'train_steps_per_second': 2.629, 'total_flos': 5879070720000.0, 'train_loss': 1.9594416300455728, 'epoch': 3.0})

In [83]:
# Save the fine-tuned model and tokenizer for future use
model_save_path = "./final_model"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)


('./final_model/tokenizer_config.json',
 './final_model/special_tokens_map.json',
 './final_model/vocab.json',
 './final_model/merges.txt',
 './final_model/added_tokens.json')

In [84]:
# Set model to evaluation mode
model.eval()

def generate_riddle(prompt):
    # Tokenize input and move tensors to the model's device (e.g., cuda)
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {key: tensor.to(model.device) for key, tensor in inputs.items()}

    outputs = model.generate(
        inputs["input_ids"],
        max_length=100,
        num_return_sequences=1,
        do_sample=True,          # enable sampling for more diversity
        temperature=1.0,         # adjust temperature for variability
        top_k=50,                # consider top 50 tokens
        top_p=0.95,              # nucleus sampling
        pad_token_id=tokenizer.eos_token_id
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

if __name__ == '__main__':
    # Get user input for the number of riddles to generate
    num_riddles = int(input("Enter the number of riddles to generate: "))

    # Get custom riddle prompt from the user (default is "Riddle:")
    custom_prompt = input("Enter your riddle prompt (press Enter to use default 'Riddle:'): ")
    if not custom_prompt.strip():
        custom_prompt = "Riddle:"

    print("\nGenerated Riddles:\n")
    for _ in range(num_riddles):
        print(generate_riddle(custom_prompt))
        print("\n" + "-" * 50 + "\n")

    # Generate and print 3 best riddles (assumed to include both riddle and answer)
    print("## Best 3 Generated Riddles:")
    best_riddles = []
    for i in range(3):
        riddle_output = generate_riddle(custom_prompt)
        best_riddles.append(riddle_output)

    for idx, riddle in enumerate(best_riddles):
        print(f"**{idx+1}. Riddle & Answer:**\n{riddle}\n")

Enter the number of riddles to generate: 3
Enter your riddle prompt (press Enter to use default 'Riddle:'): 

Generated Riddles:

Riddle: Two numbers of 12 plus three are double. What is the number?
Answer: 8

How many is it?
Answer: 20

How many times it adds up to 4?
Answer: 23

A number of 20 is 5 times 20. What am I?

Answer: 8

What is the number?

Answer: 9

Answer: 22

A number of 9 equals 16. What am I?

Answer: 32

What is the number?



--------------------------------------------------

Riddle: What number becomes 12 when you subtract 1 from it minus 5?
Answer: 9

Question: A number is divided by 3 when you multiply it by 5. What number?
Answer: 3

Answer: 9

Now you get 10. What number is that?

Answer: 17

So you get 18. What's that?

Answer: 10

I've been waiting for you to answer these questions for a while. What's your answer?

The number?

--------------------------------------------------

Riddle: If I add 10 to the square of 9 and subtract half of that number, what i